# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, broadwell)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


In [ ]:
using BenchmarkTools

M = rand(2^11, 2^11)

@btime $M * $M;

  465.359 ms (2 allocations: 32.00 MiB)


In [ ]:
try
    using CUDA
catch
    println("No GPU found.")
else
    run(`nvidia-smi`)
    # Create a new random matrix directly on the GPU:
    M_on_gpu = CUDA.CURAND.rand(2^11, 2^11)
    @btime $M_on_gpu * $M_on_gpu; nothing
end

No GPU found.


# Need Help?

* Learning: https://julialang.org/learning/
* Documentation: https://docs.julialang.org/
* Questions & Discussions:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

If you ever ask for help or file an issue about Julia, you should generally provide the output of `versioninfo()`.

In [ ]:
using Random

# Global Constants
LINES_MAX = 3
BET_MAX = 100
BET_MIN = 1

ROWS = 3
COLS = 3

# Directly using Unicode emojis as symbols on reels
heart = "❤️"
flamingo = "🦩"
diamond = "💎"
pizza = "🍕"

# Frequency of each symbols on each line

symbol_count = Dict(
    heart => 2,
    flamingo => 4,
    diamond => 6,
    pizza => 8
)

symbol_value = Dict(
    heart => 5,
    flamingo => 4,
    diamond => 3,
    pizza => 2
)

# Check if the player has won
function check_if_won(columns, lines, bet, values)
    winnings = 0
    winning_lines = Int[]
    for line in 1:lines
        symbol = columns[1][line]
        is_winning_line = true
        for column in 2:length(columns)
            symbol_to_check = columns[column][line]
            if symbol != symbol_to_check
                is_winning_line = false
                break
            end
        end
        if is_winning_line
            winnings += values[symbol] * bet
            push!(winning_lines, line)
        end
    end
    return winnings, winning_lines
end

# Get a spin for the slot machine
function get_slot_machine_spin(rows, cols, symbols)
    all_symbols = []
    for (symbol, count) in symbols
        for _ in 1:count
            push!(all_symbols, symbol)
        end
    end

    columns = [[] for _ in 1:cols]
    for col in 1:cols
        current_symbols = copy(all_symbols)
        for _ in 1:rows
            value = rand(current_symbols)
            deleteat!(current_symbols, findfirst(isequal(value), current_symbols))
            push!(columns[col], value)
        end
    end
    return columns
end

# Printing the slot machine's current state properly
function print_slot_machine(columns)
    for row in 1:length(columns[1])
        for i in 1:length(columns)
            column = columns[i]
            if i != length(columns)
                print(column[row], " | ")
            else
                print(column[row])
            end
        end
        println()
    end
end

# Function to deposit money
function deposit()
    while true
        print("Please enter the amount you would like to deposit? \$ ")
        amount = readline()
        if all(isdigit, amount)
            amount = parse(Int64, amount)
            if amount > 0
                return amount
            else
                println("Enter amount greater than zero.")
            end
        else
            println("Please enter a number.")
        end
    end
end

# Function to select the number of lines
function number_of_lines()
    while true
        println("Please enter the number of lines you would like to bet on between (1 - ", string(LINES_MAX), ") ")
        lines = readline()
        if all(isdigit, lines)
            lines = parse(Int64, lines)
            if 1 <= lines <= LINES_MAX
                return lines
            else
                println("Enter a valid number of lines")
            end
        else
            println("Please enter a number.")
        end
    end
end

# Function to select the bet amount
function bet_amount()
    while true
        print("How much would you like to bet on each line? \$ ")
        bet = readline()
        if all(isdigit, bet)
            bet = parse(Int64, bet)
            if BET_MIN <= bet <= BET_MAX
                return bet
            else
                println("Enter a valid number between (", string(BET_MIN), " - ", string(BET_MAX), ") ")
            end
        else
            println("Please enter a number")
        end
    end
end

# Main function to control the slot machine game
function main()
    balance = deposit()
    while true
        lines = number_of_lines()
        bet = bet_amount()
        total_bet = bet * lines
        if total_bet > balance
            println("You have insufficient balance. Your total balance is \$", balance)
        else
            println("You are betting \$", bet, " on ", lines, " lines. Your total bet is \$", total_bet)
            balance -= total_bet

            # Generate slot machine spin and print
            columns = get_slot_machine_spin(ROWS, COLS, symbol_count)
            print_slot_machine(columns)

            # Checking winnings
            winnings, winning_lines = check_if_won(columns, lines, bet, symbol_value)
            balance += winnings
            if winnings > 0
                println("Congratulations! You won \$", winnings, " on lines ", winning_lines)
            else
                println("Sorry, No luck this time.")
            end

            println("Your current balance is \$", balance)

            # Checking if player wants to play again
            print("Would you like to play again? (y/n) ")
            response = readline()
            if response != "y"
                break
            end
        end
    end
end

main()



Please enter the amount you would like to deposit? $ stdin> 1000
Please enter the number of lines you would like to bet on between (1 - 3) 
stdin> 3
How much would you like to bet on each line? $ stdin> 100
You are betting $100 on 3 lines. Your total bet is $300
❤️ | 🦩 | 💎
💎 | ❤️ | 🍕
💎 | 🍕 | 🍕
Sorry, No luck this time.
Your current balance is $700
Would you like to play again? (y/n) stdin> y
Please enter the number of lines you would like to bet on between (1 - 3) 
stdin> 3
How much would you like to bet on each line? $ stdin> 50
You are betting $50 on 3 lines. Your total bet is $150
🍕 | 🍕 | 🍕
🍕 | 🦩 | 🦩
🍕 | 🍕 | 🦩
Congratulations! You won $100 on lines [1]
Your current balance is $650
Would you like to play again? (y/n) stdin> y
Please enter the number of lines you would like to bet on between (1 - 3) 
stdin> 2
How much would you like to bet on each line? $ stdin> 100
You are betting $100 on 2 lines. Your total bet is $200
🦩 | 🍕 | 🍕
🦩 | 🍕 | 💎
🦩 | 💎 | 🍕
Sorry, No luck this time.
Your cur

Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />